# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} x_1 ~~ s.t. ~ x_1 + x_2 + x_3 = 1 ~, ~~ x \ge 0$

Dual: $\underset{\lambda \in \mathbb{R}, s \in \mathbb{R^3}}{\max} \lambda ~~ s.t. ~ [1~1~1]' \lambda + s = [1~0~0]'~, ~~s\ge0$

### 2)

## 1) Short-Step Path-Following 

In [1]:
using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures
using QuadraticModels
using Printf
using SparseArrays

In [2]:
function primal_dual_diff(c, b, x, lambda)
    return abs(c'*x - b'*lambda) / (1 + abs(c'*x))
end

primal_dual_diff (generic function with 1 method)

In [19]:
"""
           STPF1(c, A, b, init_vect, max_iter, tol_step_x, eps, theta)
        Compute the optimisation of the linear problem min `c`'x, s.t. `A`x = `b` (Primal), 
        with the Short-Step Path Following method.
        
        We have the Dual problem: max `b`'λ, s.t. `A`'λ + s = c.

        `init_vect` shall be the initial vector ``(x_0,λ_0, s_0)``.
        
        `max_iter` is the maximum number of iterations.

        `tol_step_x` is the minimum tolerance of the difference between two successive iterates of x.

        `eps` is the measure of smallness.

        `theta` is the parameter such that ``||XSe- μe||_2 ≤ theta*μ ``, 

            where μ = s'x/length(x), X = Diagonal(x), S = Diagonal(s), and e = ones(1, length(x)).
       """
function STPF1(QM, s0, max_iter, tol_step_x= 1e-4, eps_pdd=1e-4, eps_mu=1e-4, theta=0.4)
    
    # get variables from QuadraticModel
    A = sparse(QM.data.Arows, QM.data.Acols, QM.data.Avals)
    H = sparse(QM.data.Hrows, QM.data.Hcols, QM.data.Hvals)
    c = QM.data.c
    c0 = QM.data.c0
    x_k, lambda_k, s_k = QM.meta.x0, QM.meta.y0, s0
    @assert QM.meta.lcon == QM.meta.ucon # equality constraint (Ax=b)
    b = QM.meta.lcon
    
    # necessary conditions to apply the algorithm (strictly feasible starting point)
    @assert all(x_k.>0) && all(s_k.>0)
    @assert all(A*x_k - b .< 1e-16)
    
    n_rows, n_cols = size(A) 
    sigma = 1 - theta / sqrt(n_cols)
    k = 0 # iter
    e = ones(n_cols, 1)
    
    # control variables
    step_x = 1.0+tol_step_x
    pdd = primal_dual_diff(c, b, x_k, lambda_k)
    mu_k = s_k' * x_k / n_cols
    
    # display
    println("Iter | primal_objective | primal-dual difference |     step x     |     mu")
    println("-------------------------------------------------------------------------------------")
    @printf("% 4d |     % 7.2e    |        % 7.2e       |   % 7.2e    | % 7.2e\n", k, c'*x_k, pdd, step_x, mu_k)
    
    while k <= max_iter && step_x > tol_step_x && pdd > eps_pdd && mu_k > eps_mu 
        Xk = Diagonal(x_k)
        Sk = Diagonal(s_k)
        # Newton's method:
        Jacob_Fk = [zeros(n_cols, n_cols) A' I
                    A zeros(n_rows, n_rows) zeros(n_rows, n_cols)
                    Sk zeros(n_cols, n_rows) Xk]
        Fk = [zeros(n_rows+n_cols, 1)
              -x_k.*s_k .+ sigma*mu_k]
        dir_k = Jacob_Fk\Fk # error if Jacob_Fk is singular
        
        #update:
        x_k_prec = copy(x_k)
        x_k += dir_k[1:n_cols]
        lambda_k += dir_k[n_cols+1: n_cols+n_rows]
        s_k += dir_k[n_cols+n_rows+1:end]
        
        
        mu_k = s_k' * x_k / n_cols
        step_x = norm(x_k - x_k_prec)
        
        pdd = primal_dual_diff(c, b, x_k, lambda_k)
        k += 1
        
        # affichage
        @printf("% 4d |     % 7.2e    |        % 7.2e       |   % 7.2e    | % 7.2e\n", k, c0+c'*x_k, pdd, step_x, mu_k)
    end
    
    criteria = [k > max_iter, pdd <= eps_pdd,  step_x <= tol_step_x, mu_k <= eps_mu]
    criteria_names = ["reached max_iter", "pdd <= eps_pdd", "step_x <= tol_step_x"," mu_k <= eps_mu"]
    println("\n stopping criterion = ",criteria_names[findall(criteria)])
    return OrderedDict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_opt" => s_k, 
        "n_iter" => k, "pdd" => pdd, "step_x" => step_x, "mu" => mu_k)
end



function display_results(result)
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

    

display_results (generic function with 1 method)

In [20]:
### problem 1
c = [1; 0; 0]
c01 = 0.
A = [1 1 1]
b = [1]
lvar = [0;0;0]
uvar = [Inf; Inf; Inf];
lcon = b;
ucon = b;

# initialisation 1 
x01 = [1/3, 1/3, 1/3]
lambda01 = [-1]
s01 = [2, 1, 1]
init_vect1 = (x01, lambda01, s01);

### bibliothèque QuadraticModels

optimize $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $~~L \le Ax \le U$ and $l \le x \le u$ 

Ici L = lcon, U = ucon, l = lvar, u = uvar

In [21]:
# creation QuadraticModel object
QM = QuadraticModel(c, zeros(3,3), A=A, lcon=lcon, ucon=ucon, lvar=lvar, uvar=uvar, x0=x01, c0=c01, name="QM1")
Acols = QM.data.Acols
Arows = QM.data.Arows
Avals = QM.data.Avals
newA = sparse(Arows, Acols, Avals)
#newA = similar(undef, (Arows, Avals))
QM.meta.y0 #lambda0 

1-element Array{Float64,1}:
 0.0

In [22]:
res_opt1 = STPF1(QM, s01, 100);
display_results(res_opt1)

Iter | primal_objective | primal-dual difference |     step x     |     mu
-------------------------------------------------------------------------------------
   0 |      3.33e-01    |         2.50e-01       |    1.00e+00    |  4.44e-01
   1 |      2.00e-01    |         2.12e-02       |    1.63e-01    |  3.42e-01
   2 |      1.66e-01    |         1.81e-01       |    4.13e-02    |  2.63e-01
   3 |      1.41e-01    |         3.45e-01       |    3.14e-02    |  2.02e-01
   4 |      1.17e-01    |         4.78e-01       |    2.86e-02    |  1.55e-01
   5 |      9.62e-02    |         5.85e-01       |    2.59e-02    |  1.20e-01
   6 |      7.77e-02    |         6.72e-01       |    2.26e-02    |  9.20e-02
   7 |      6.21e-02    |         7.42e-01       |    1.91e-02    |  7.07e-02
   8 |      4.92e-02    |         7.98e-01       |    1.58e-02    |  5.44e-02
   9 |      3.88e-02    |         8.42e-01       |    1.28e-02    |  4.18e-02
  10 |      3.03e-02    |         8.77e-01       |    1.03e

OrderedDict{String,Any} with 7 entries:
  "x_opt"      => [0.000219023, 0.49989, 0.49989]
  "lambda_opt" => [0.999562]
  "s_opt"      => [1.00044, 0.000438307, 0.000438307]
  "n_iter"     => 29
  "pdd"        => 0.999124
  "step_x"     => 8.05098e-5
  "mu"         => 0.00021911

## 2) Predictor Corrector

In [265]:
function Point_in_Neighborhood2(theta, x, s, mu, e)
    return norm(x.*s .- mu*e) <= theta*mu  # faut-il tester x>0, s>0 ?
end



function Next_Point_in_Neighborhood2(alpha_step, theta, x, s, dir_k, mu, e, n, m)
    x_next = x + alpha_step*dir_k[1:n]
    s_next = s + alpha_step*dir_k[n+m+1: end]
    mu_next = s_next' * x_next / n
    return Point_in_Neighborhood2(theta, x_next, s_next, mu_next, e)
end



function Alpha_PredictorStep(theta, x, lambda, s, dir_k, mu, e, alpha_step)
    alpha = 0
    x_k, lambda_k, s_k = x, lambda, s
    n = length(x)
    m = length(lambda)
    mu_k = s_k' * x_k / n
    while Next_Point_in_Neighborhood2(alpha_step, theta, x_k, s_k, dir_k, mu_k, e, n, m) == true && 
                alpha+alpha_step <= 1
        alpha += alpha_step
        x_k += alpha_step * dir_k[1:n]
        lambda_k += alpha_step * dir_k[n+1: n+m]
        s_k += alpha_step * dir_k[n+m+1: end]
        mu_k = s_k' * x_k / n
    end
    return x_k, lambda_k, s_k        
end

Alpha_PredictorStep (generic function with 1 method)

In [312]:
"""
           PredictorCorrector1(c, A, b, init_vect, max_iter, eps, theta)
       Compute the optimisation of the linear problem min `c`'x, s.t. `A`x = `b` (Primal), 
        with the Predictor- Corrector method.
        
        We have the Dual problem: max `b`'λ, s.t. `A`'λ + s = c.

        `init_vect` shall be the initial vector ``(x_0,λ_0, s_0)``.
        
        `max_iter` is the maximum number of iterations.

        `eps` is the measure of smallness.

        `theta1` is the parameter for the Predictor Step such that ``||XSe- μe||_2 ≤ theta*μ ``, 

            where μ = s'x/length(x), X = Diagonal(x), S = Diagonal(s), and e = ones(1, length(x)) for the inner neighborhood.

       """
function PredictorCorrector1(c, A, b, init_vect, max_iter, eps_pdd = 1e-4, eps_mu=1e-4,
        theta=0.5, alpha_step=1e-3)
    
    x_k, lambda_k, s_k = init_vect
    n_rows, n_cols = size(A) 
    mu_k = s_k' * x_k / n_cols
    k = 0
    pdd = 1.0
    e = ones(n_cols, 1)
    
    # affichage
    println("Iter | primal_objective | primal-dual difference |     mu")
    println("-------------------------------------------------------------------------------")
    @printf("% 4d |     % 7.2e    |        % 7.2e       |   % 7.2e\n", k, c'*x_k, pdd, mu_k)
    
    while k <= max_iter && pdd > eps_pdd && mu_k > eps_mu
        Xk = Diagonal(x_k)
        Sk = Diagonal(s_k)
        
        Jacob_Fk = [zeros(n_cols, n_cols) A' I
                    A zeros(n_rows, n_rows) zeros(n_rows, n_cols)
                    Sk zeros(n_cols, n_rows) Xk]
        Fk = [zeros(n_rows+n_cols, 1)
                -x_k.*s_k]
        if k%2 == 0           # predictor step, sigma=0

            dir_k = Jacob_Fk\Fk
            x_k, lambda_k, s_k = Alpha_PredictorStep(theta, x_k, lambda_k, s_k, dir_k, mu_k, e, alpha_step)
        else
            Fk += [zeros(n_rows+n_cols, 1)
                   mu_k*e ]
            dir_k = Jacob_Fk\Fk

            x_k += dir_k[1:n_cols]
            lambda_k += dir_k[n_cols+1: n_cols+n_rows]
            s_k += dir_k[n_cols+n_rows+1:end]
        end
        
        mu_k = s_k' * x_k / n_cols
        pdd = primal_dual_diff(c, b, x_k, lambda_k)
        k += 1
        
        # affichage
        @printf("% 4d |     % 7.2e    |        % 7.2e       |   % 7.2e\n", k, c'*x_k, pdd, mu_k)
        
    end
    
    criteria = [k > max_iter, pdd <= eps_pdd, mu_k <= eps_mu]
    criteria_names = ["reached max_iter", "pdd <= eps_pdd", "mu_k <= eps_mu"]
    println("\n stopping criterion = ",criteria_names[findall(criteria)])
    return OrderedDict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_opt" => s_k, 
        "n_iter" => k, "pdd" => pdd, "mu" => mu_k)
end

PredictorCorrector1

In [313]:
### probleme 1
c = [1, 0 ,0]
A = Matrix([1, 1, 1]')
b = [1];

In [314]:
# initialisation 1 
x01 = [1/3, 1/3, 1/3]
lambda01 = [-1]
s01 = [2, 1, 1]
init_vect1 = (x01, lambda01, s01);
res_pc1 = PredictorCorrector1(c, A, b, init_vect1, 100);
display_results(res_pc1)

Iter | primal_objective | primal-dual difference |     mu
-------------------------------------------------------------------------------
   0 |      3.33e-01    |         1.00e+00       |    4.44e-01
   1 |      3.33e-01    |         1.00e+00       |    4.44e-01
   2 |      2.00e-01    |         1.11e+00       |    4.44e-01
   3 |      1.40e-01    |         3.20e-01       |    1.22e-01
   4 |      9.42e-02    |         3.34e-01       |    1.22e-01
   5 |      3.89e-02    |         8.37e-02       |    2.90e-02
   6 |      2.72e-02    |         8.46e-02       |    2.90e-02
   7 |      4.29e-03    |         9.18e-03       |    3.07e-03
   8 |      3.05e-03    |         9.19e-03       |    3.07e-03
   9 |      6.43e-05    |         1.38e-04       |    4.61e-05

 stopping criterion = ["mu_k <= eps_mu"]

-----------------------------------------------------------------------
------------------------------- RESULTS -------------------------------


OrderedDict{String,Any} with 6 entries:
  "x_opt"      => [6.42521e-5, 0.499968, 0.499968]
  "lambda_opt" => [-7.39973e-5]
  "s_opt"      => [1.00007, 7.39973e-5, 7.39973e-5]
  "n_iter"     => 9
  "pdd"        => 0.000138241
  "mu"         => 4.60831e-5